In [1]:
import os
import tools
import numpy as np
import pandas as pd
import scanpy as sc

sc.logging.print_header()

scanpy==1.8.1 anndata==0.7.6 umap==0.5.1 numpy==1.20.2 scipy==1.6.2 pandas==1.2.4 scikit-learn==0.24.2 statsmodels==0.12.2 python-igraph==0.9.1 pynndescent==0.5.2


In [2]:
os.chdir('/lustre/scratch117/cellgen/team297/kt16/COVID_imperial_renal')

In [3]:
bdata = sc.read_h5ad('h5ad/df.fil3_gex_bcells_vdj.h5ad')
bdata

AnnData object with n_obs × n_vars = 75683 × 385
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'nCount_ADT_raw', 'nFeature_ADT_raw', 'nCount_RNA_raw', 'nFeature_RNA_raw', 'soupxOnAdt', 'soupxOnRna', 'percentMito', 'exclude', 'centre', 'pool', 'tcrId', 'bcrId', 'celltypist_broad_predLabel', 'celltypist_broad_maxPredProb', 'haniffa_broad_predLabel', 'haniffa_broad_maxPredProb', 'yoshida_broad_predLabel', 'yoshida_broad_maxPredProb', 'celltypist_detailed_predLabel', 'celltypist_detailed_maxPredProb', 'yoshida_detailed_predLabel', 'yoshida_detailed_maxPredProb', 'seurat_clusters', 'RNA_sequencing_saturation', 'pool_factor', 'RNA_readsConfidentlyOnTranscriptome', 'RNA_readsConfidentlyOnGenome', 'RNA_readsOnGenome', 'RNA_Q30readsBarcode', 'RNA_Q30readsRead', 'RNA_Q30readsUmi', 'rna_snn_res.4', 'rna_snn_res.0.5', 'rna_snn_res.0.1', 'harmony_snn_res.4', 'yoshida_broad_predLabel_majorVote', 'harmony_snn_res.0.5', 'harmony_snn_res.0.1', 'cell_type_compartment

In [4]:
# trimm the columns
bdata.obs = bdata.obs[['orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'nCount_ADT_raw', 'nFeature_ADT_raw', 'nCount_RNA_raw', 'nFeature_RNA_raw', 'soupxOnAdt', 'soupxOnRna', 'percentMito', 'exclude', 'centre', 'pool', 'tcrId', 'bcrId', 'celltypist_broad_predLabel', 'celltypist_broad_maxPredProb', 'haniffa_broad_predLabel', 'haniffa_broad_maxPredProb', 'yoshida_broad_predLabel', 'yoshida_broad_maxPredProb', 'celltypist_detailed_predLabel', 'celltypist_detailed_maxPredProb', 'yoshida_detailed_predLabel', 'yoshida_detailed_maxPredProb', 'seurat_clusters', 'RNA_sequencing_saturation', 'pool_factor', 'merged_souporcell_status', 'patient_id', 'sample_id', 'ethnicity', 'sex', 'calc_age', 'ihd', 'previous_vte', 'copd', 'diabetes', 'smoking', 'cause_eskd', 'renal_status', 'WHO_severity', 'admission_date', 'discharge_date', 'fatal_disease', 'case_control', 'date_positive_swab', 'date_first_symptoms', 'radiology_evidence_covid', 'sample_date', 'temp_WCC', 'temp_neut', 'temp_mono', 'temp_lymph', 'temp_CRP', 'temp_ddimer', 'temp_ferritin', 'temp_trop', 'WHO_temp_severity', 'individual_id', 'celltype_B_corrected']]
# remove doublets
bdata = bdata[bdata.obs['merged_souporcell_status'] == 'singlet'].copy()
# remove the unannotated samples
bdata = bdata[(bdata.obs['sample_id'] != 'NA') & (~bdata.obs['patient_id'].isin(['NULL', 'unresolved'])) & (bdata.obs['case_control'] != 'NA')].copy()
bdata.n_obs

66689

In [5]:
# calculate days to sampling, days to admission, days to first symptoms, days to discharge from positive swab
tmp = bdata.copy()
tmp.obs['sample_date'] = tmp.obs['sample_date'].astype('object')
tmp.obs['admission_date'] = tmp.obs['admission_date'].astype('object')
tmp.obs['date_positive_swab'] = tmp.obs['date_positive_swab'].astype('object')
tmp.obs['discharge_date'] = tmp.obs['discharge_date'].astype('object')
tmp.obs['date_first_symptoms'] = tmp.obs['date_first_symptoms'].astype('object')
tmp.obs['date_first_symptoms'].replace({'NA':np.nan}, inplace = True)
tmp.obs['admission_date'].replace({'NA':np.nan}, inplace = True)
tmp.obs['date_positive_swab'].replace({'NA':np.nan}, inplace = True)
tmp.obs['discharge_date'].replace({'NA':np.nan}, inplace = True)
days_to_sampling = pd.to_datetime(tmp.obs['sample_date'], format='%d/%m/%Y') - pd.to_datetime(tmp.obs['date_positive_swab'], format='%d/%m/%Y')
days_to_admission = pd.to_datetime(tmp.obs['admission_date'], format='%d/%m/%Y') - pd.to_datetime(tmp.obs['date_positive_swab'], format='%d/%m/%Y')
days_to_firstsymptoms = pd.to_datetime(tmp.obs['date_first_symptoms'], format='%d/%m/%Y') - pd.to_datetime(tmp.obs['date_positive_swab'], format='%d/%m/%Y')
days_to_discharge = pd.to_datetime(tmp.obs['discharge_date'], format='%d/%m/%Y') - pd.to_datetime(tmp.obs['date_positive_swab'], format='%d/%m/%Y')
days_to_sampling = days_to_sampling.dt.days
days_to_admission = days_to_admission.dt.days
days_to_firstsymptoms = days_to_firstsymptoms.dt.days
days_to_discharge = days_to_discharge.dt.days
days_to_sampling.fillna(0, inplace = True)
days_to_admission.fillna(0, inplace = True)
days_to_firstsymptoms.fillna(0, inplace = True)
days_to_discharge.fillna(0, inplace = True)
bdata.obs['days_to_sampling'] = days_to_sampling
bdata.obs['days_to_admission'] = days_to_admission
bdata.obs['days_to_firstsymptoms'] = days_to_firstsymptoms
bdata.obs['days_to_discharge'] = days_to_discharge

/nfs/users/nfs_k/kt16/miniconda2/envs/dandelion/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: modifications to a property of a datetimelike object are not supported and are discarded. Change values on the original.
  return super().fillna(


In [6]:
bdata.write('h5ad/df.fil3_gex_bcells_vdj_trimmed.h5ad', compression = 'gzip')

### Setup for milo

anndata2ri converter doesn't seem to like converting anndata objects storing the KNN graph in the adata.obsp slot, probably because there is no dedicated slot in the SingleCellExperiment object. Because of this, we will first make a copy of our anndata without KNN graph for conversion.

In [7]:
# store the neighborhood graph separately
adata_no_knnx = bdata.copy()
adata_no_knnx.obsp = None
adata_no_knnx.uns.pop("neighbors")

## Save the binary connectivity matrix
knn_adjacencyx = bdata.obsp["connectivities"]

use the anndata2ri package to allow conversion between anndata and SingleCellExperiment objects

In [8]:
import anndata2ri
import rpy2.rinterface_lib.callbacks
import logging

In [9]:
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

In [10]:
anndata2ri.activate()

In [11]:
%load_ext rpy2.ipython

In [12]:
%%R
## Count cells in neighbourhoods
countCells_ <- function(x, samples, meta.data=NULL){

    # cast dplyr objects to data.frame
    if(!is.data.frame(meta.data) & !is.null(meta.data)){
        meta.data <- as.data.frame(meta.data)
    }

    if(length(samples) > 1 & !is.null(meta.data)){
        stop("Multiple sample columns provided, please specify a unique column name")
    } else if(is.null(meta.data) & length(samples) != ncol(x)){
        stop(paste0("Length of vector does not match dimensions of object. Length:",
                    length(samples), " Dimensions: ", ncol(x)))
    }

    # check the nhoods slot is populated
    if(ncol(nhoods(x)) == 1 & nrow(nhoods(x)) == 1){
        stop("No neighbourhoods found. Please run makeNhoods() first.")
    }

    message("Checking meta.data validity")
    if(!is.null(meta.data)){
        samp.ids <- unique(as.character(meta.data[, samples]))
    } else {
        samp.ids <- unique(as.character(samples))
    }

    num.hoods <- ncol(nhoods(x))

    ## Convert meta data to binary dummies in sparse matrix
    dummy.meta.data <- Matrix(data=0, nrow=nrow(meta.data), ncol = length(samp.ids), sparse = TRUE)
    colnames(dummy.meta.data) <- samp.ids
    rownames(dummy.meta.data) <- rownames(meta.data)
    for (s in seq_along(samp.ids)){
        i.s <- samp.ids[s]
        s.ixs <- which(meta.data[samples]==i.s)
        dummy.meta.data[s.ixs, as.character(i.s)] <- 1
    }

    message("Counting cells in neighbourhoods")
    count.matrix <- Matrix::t(nhoods(x)) %*% dummy.meta.data

    # add to the object
    rownames(count.matrix) <- c(1:num.hoods)
    nhoodCounts(x) <- count.matrix

    return(x)
}

In [13]:
%%R -i adata_no_knnx -i knn_adjacencyx
save(countCells_, adata_no_knnx, knn_adjacencyx, file = 'h5ad/df.fil3_gex_bcells_vdj_trimmed_milo_prep.RData')